In [1]:
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv(r"C:\Users\Carlos\Documents\Python Projects\Logistical Regression\Predict Credit Card Fraud\transactions.csv")

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


In [4]:
df_corr=df.corr().abs()
df_corr

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
step,1.000000,0.020598,0.012566,0.012849,0.026898,0.024205,0.034771
amount,0.020598,1.000000,0.001412,0.006926,0.321497,0.488105,0.080721
oldbalanceOrg,0.012566,0.001412,1.000000,0.998737,0.068276,0.042432,0.012824
newbalanceOrig,0.012849,0.006926,0.998737,1.000000,0.069920,0.042345,0.007229
oldbalanceDest,0.026898,0.321497,0.068276,0.069920,1.000000,0.974219,0.006936
newbalanceDest,0.024205,0.488105,0.042432,0.042345,0.974219,1.000000,0.000807
isFraud,0.034771,0.080721,0.012824,0.007229,0.006936,0.000807,1.000000


In [5]:
df_corr=df_corr[df_corr!=1.0]
df_corr.unstack().sort_values(ascending=False)[:20]


newbalanceOrig  oldbalanceOrg     0.998737
oldbalanceOrg   newbalanceOrig    0.998737
oldbalanceDest  newbalanceDest    0.974219
newbalanceDest  oldbalanceDest    0.974219
                amount            0.488105
amount          newbalanceDest    0.488105
oldbalanceDest  amount            0.321497
amount          oldbalanceDest    0.321497
                isFraud           0.080721
isFraud         amount            0.080721
oldbalanceDest  newbalanceOrig    0.069920
newbalanceOrig  oldbalanceDest    0.069920
oldbalanceOrg   oldbalanceDest    0.068276
oldbalanceDest  oldbalanceOrg     0.068276
oldbalanceOrg   newbalanceDest    0.042432
newbalanceDest  oldbalanceOrg     0.042432
                newbalanceOrig    0.042345
newbalanceOrig  newbalanceDest    0.042345
isFraud         step              0.034771
step            isFraud           0.034771
dtype: float64

In [6]:
df['isPayment']=0
df['isPayment']=df['type'].isin(['PAYMENT', 'DEBIT']).astype(int) # easy way to add 2 variables as "true" and change "true" to 1 with ".astype(int)" 
df['isPayment'].head()

0    0
1    0
2    0
3    0
4    0
Name: isPayment, dtype: int32

In [7]:
df['isMovement']=0
df['isMovement']=df['type'].isin(['CASH_OUT','TRANFER']).astype(int)
df['isMovement'].head()

0    1
1    1
2    0
3    1
4    1
Name: isMovement, dtype: int32

In [8]:
df['accountDiff']=abs(df['oldbalanceOrg']-df['oldbalanceDest'])

In [9]:
df_corr=df.corr().abs()

In [10]:
df_corr=df_corr[df_corr!=1.0]
df_corr.unstack().sort_values(ascending=False)[:20]

newbalanceOrig  oldbalanceOrg     0.998737
oldbalanceOrg   newbalanceOrig    0.998737
newbalanceDest  oldbalanceDest    0.974219
oldbalanceDest  newbalanceDest    0.974219
                accountDiff       0.784128
accountDiff     oldbalanceDest    0.784128
newbalanceDest  accountDiff       0.759232
accountDiff     newbalanceDest    0.759232
                oldbalanceOrg     0.590663
oldbalanceOrg   accountDiff       0.590663
accountDiff     newbalanceOrig    0.589085
newbalanceOrig  accountDiff       0.589085
isMovement      isPayment         0.535190
isPayment       isMovement        0.535190
amount          newbalanceDest    0.488105
newbalanceDest  amount            0.488105
oldbalanceDest  amount            0.321497
amount          oldbalanceDest    0.321497
accountDiff     isPayment         0.269001
isPayment       accountDiff       0.269001
dtype: float64

In [11]:
df.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isPayment,isMovement,accountDiff
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0,0,1,474016.32
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0,0,1,2720411.37
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0,0,0,7885498.11


In [12]:
x=df[['amount','isPayment','isMovement','accountDiff']]
y=df['isFraud']

In [13]:
x_train, x_test, y_train, y_test=train_test_split(x,y)

In [14]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [15]:
model=LogisticRegression().fit(x_train, y_train)

In [16]:
model.score(x_test,y_test) # test scores

0.99818

In [17]:
model.score(x_train,y_train) # training scores

0.998706658044387

In [18]:
# new transaction data
transaction1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transaction2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transaction3 = np.array([543678.31, 1.0, 0.0, 510025.5])

In [19]:
#stacking the new transactions into one df
all_transactions=np.stack((transaction1,transaction2,transaction3)) # have to add the extra parenthesis for this to work

In [20]:
trans_scaled=scaler.transform(all_transactions)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [21]:
model.predict(trans_scaled) # our predictions are all 0 which means it's not fraudulent

array([0, 0, 0], dtype=int64)

In [22]:
model.predict_proba(trans_scaled)

array([[9.97973147e-01, 2.02685263e-03],
       [9.99977823e-01, 2.21766705e-05],
       [9.99976006e-01, 2.39943356e-05]])